# Hyper-Parameter Tuning for BaggedKNN

In [ ]:
!pip install scikit-optimize
!pip install scikit-learn
!pip install pandas

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV

In [ ]:
x = "your/eeg_features/as/array"
groups = "your/groups/as/array"
y = "your/subject_ages/as/array"

In [ ]:
y_skf = [int(age) for age in y]
skf_idx = []
skf = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=126)
for fold, (train_index, test_index) in enumerate(skf.split(x, y_skf, groups)):
    skf_idx.append((train_index, test_index))

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
model = KNeighborsRegressor(n_jobs=-2)

In [ ]:
parameter_space = {
    'n_neighbors': [3, 50],
    'leaf_size': [3, 100],
    'p': [1, 2]
}

clf = BayesSearchCV(estimator=model,
                    search_spaces=parameter_space,
                    cv=skf_idx,
                    scoring='neg_mean_absolute_error',
                    verbose=4)

clf.fit(x, y=y)

In [ ]:
print(clf.cv_results_)
print(clf.best_score_)
print(clf.best_params_)
results = pd.DataFrame(clf.cv_results_)